# Example of using `cells_in_gel` in early stages :) 

## Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from skimage import io

from cells_in_gel import preprocess as pp
from cells_in_gel import properties as props

## Load Image

Load a stack or single fluorescence microcopy image from the phalloidin 488 channel. If you have a stack of images, it is helpful to use `io.imread` and call a specific stack within the stack as shown below.

In [ ]:
im_stack = io.imread('C3-NTG-CFbs_NTG5ECM_1mMRGD_20x_003.tif')
im = im_stack[10] # image 10 from the stack

## Segmentation

The `cells_in_gell.preprocess.phalloidin_488_segment` function binarizes your image and identifies objects within the image as well. The `min_size` feature allows removal of objects within a specified size. The `label` returns a labeled image while `overlay` returns an overlay of identified objects.

In [ ]:
label, overlay = pp.phalloidin_488_segment(im, min_size=350)

Plotting the original image next to the overlayed image shows the binarization and object identification.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 7))

axes[0].imshow(im, cmap='gray')
axes[0].set_title('Original', fontsize='20')

axes[1].imshow(overlay, cmap='gray')
axes[1].set_title('Overlay of Objects', fontsize='20')

for ax in axes:
    ax.axis('off')

## Properties

Extracting properties from objects in the image such as area, eccentricity, extent, coordinates, etc. can be done using the `cells_in_gels.properties.im_properties` function. The input calls for the labeled image we created in the segmentation section and the orignal image shown below. 

In [ ]:
regions = props.im_properties(label, im)

Here is what the pandas dataframe looks like!

In [ ]:
regions.head()

If you wish to see a distribution of various properties in the image, simply plot a histogram as shown below.

In [ ]:
regions.hist(column='area')

The total area of this channel can also be calculated using the sum feature.

In [ ]:
total_phalloidin = sum(regions.area)
total_phalloidin

## Multiple Images

It might also be useful to perform these operations on mutliple images and determine region areas of the entire stack. First, find the dimensions of your stack, and create empty arrays for the lable and overlay images.  

In [ ]:
im_stack = io.imread('C3-NTG-CFbs_NTG5ECM_1mMRGD_20x_003.tif')
im = im_stack[10] # image 10 from the stack

label, overlay = pp.phalloidin_488_segment(im, min_size=350)

In [ ]:
x, y, z = im_stack.shape # dimensions of the image stack
label_stack = np.zeros_like(im_stack) # create emtpy array for label
overlay_stack = np.zeros((x, y, z, 3)) # create emtpy array for overlay

Apply the `cells_in_gel.preprocessing.phalloidin_488_segment` function to the stack. 

In [ ]:
for i in range(x):
    label_stack[i], overlay_stack[i] = pp.phalloidin_488_segment(im_stack[i], min_size=350)

Plot a slice of the original image and the overlay of objects to see if it worked!

In [ ]:
# slice 1
fig, axes = plt.subplots(ncols=2, figsize=(12, 7))

axes[0].imshow(im_stack[1], cmap='gray')
axes[0].set_title('Original', fontsize='20')

axes[1].imshow(overlay_stack[1], cmap='gray')
axes[1].set_title('Overlay of Objects', fontsize='20')

for ax in axes:
    ax.axis('off')

Now create an emtpy dictionary to collect dataframes for each image slice. 

In [ ]:
frames = {}
groups = [i for i in range(x)]

Apply the `cells_in_gel.properties.im_properties` function to the entire stack. 

In [ ]:
for grp in groups:
    frames[grp] = props.im_properties(label_stack[grp], im_stack[grp])

Take a look at one of the slice dataframes to check it out!

In [ ]:
frames[0].head()

Now sum the total area for each slice in the stack and place into a new pandas dataframe named `area_stack`.

In [ ]:
total_area = np.zeros(x)

for grp in groups:
    total_area[grp] = sum(frames[grp].area)

In [ ]:
area_stack = pd.DataFrame(total_area, groups)

In [ ]:
area_stack.rename(columns={0: "Area"})

Woohoo!! There you have it! :) Area of objects in each slice.